<a href="https://colab.research.google.com/github/armandossrecife/mysummary/blob/main/testes_extracao_dados_html.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usando o Beautifulsoup

https://beautiful-soup-4.readthedocs.io/en/latest/

https://en.wikipedia.org/wiki/Beautiful_Soup_(HTML_parser)

Parse tree

O processo de web scraping


## Instalando o Beautifulsoap

In [190]:
!pip install beautifulsoup4

## Recuperando informações com o Beautifulsoap

In [191]:
from bs4 import BeautifulSoup
import requests

In [192]:
# Fetch the HTML content of the webpage
MY_URL = "https://en.wikipedia.org/wiki/Linux"
html_content = ""

try:
  response = requests.get(MY_URL)
  response.raise_for_status()  # Verifica se a requisição foi bem-sucedida

  if response.status_code == 200:
      print("Requisição bem sucedida!")
      html_content = response.content
  else:
      print("Requisição falhou com o código de status:", response.status_code)
except requests.exceptions.RequestException as e:
    print("Error fetching the webpage:", e)

Requisição bem sucedida!


In [215]:
# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# <div id="mw-content-text"
body_content_div = soup.find('div', id="mw-content-text")

In [216]:
# Extract the text content
if body_content_div:
    content = body_content_div.get_text(strip=True)
    fragmento = content[:1000]
    print(fragmento)
else:
    print("Div with id 'bodyContent' not found.")

Family of Unix-like operating systemsThis article is about the family of operating systems. For the kernel, seeLinux kernel. For other uses, seeLinux (disambiguation).Operating systemLinuxTuxthe penguin, the mascot of Linux[1]DeveloperCommunity contributors,Linus TorvaldsWritten inC,assembly languages,Rustand othersOS familyUnix-likeWorking stateCurrentSource modelOpen sourceInitial releaseSeptember 17, 1991; 33 years ago(1991-09-17)Repositorygit.kernel.org/pub/scm/linux/kernel/git/torvalds/linux.git/github.com/torvalds/linuxMarketing targetCloud computing,embedded devices,mainframe computers,mobile devices,personal computers,servers,supercomputersAvailable inMultilingualPlatformsAlpha,ARC,ARM,C-Sky,Hexagon,LoongArch,m68k,Microblaze,MIPS,Nios II,OpenRISC,PA-RISC,PowerPC,RISC-V,s390,SuperH,SPARC,x86,XtensaKerneltypeMonolithicUserlandGNU,[a]BusyBox[b]Defaultuser interfaceUnix shell(CLI)Mostdistributionsinclude adesktop environment(GUI).LicenseGPLv2[9][c]Official websitekernel.orgArticles

## Recuperando os dados de um protocolo

In [217]:
class DocumentoProcesso:
  def __init__(self, tipo, data, origem, natureza, link):
    self.tipo = tipo
    self.data = data
    self.origem = origem
    self.natureza = natureza
    self.link = link

  def __str__(self):
    return f"Tipo: {self.tipo}, Data: {self.data}, Origem: {self.origem}, Natureza: {self.natureza}, Link: {self.link}"

class MovimentacaoProcesso:
  def __init__(self, data_origem, unidade_origem, unidade_destino, enviado_por, recebido_em, recebido_por):
    self.data_origem = data_origem
    self.unidade_origem = unidade_origem
    self.unidade_destino = unidade_destino
    self.enviado_por = enviado_por
    self.recebido_em = recebido_em
    self.recebido_por = recebido_por

  def __str__(self):
    return f"Data Origem: {self.data_origem}, unidade origem: {self.unidade_origem}, unidade destino: {self.unidade_destino}, enviado por: {self.enviado_por}, recebido em: {self.recebido_em}, recebido por: {self.recebido_por}"

In [218]:
def get_html_content(url):
  html_content = ""
  try:
    response = requests.get(url)
    response.raise_for_status()  # Verifica se a requisição foi bem-sucedida

    if response.status_code == 200:
        print("Requisição bem sucedida!")
        html_content = response.content
    else:
        raise ValueError("Requisição falhou com o código de status:", response.status_code)
    return html_content
  except requests.exceptions.RequestException as e:
      raise ValueError("Error fetching the webpage:", e)

In [219]:
URL_PROTOCOLO = "https://www.sipac.ufpi.br/public/jsp/processos/processo_detalhado.jsf?id=594067"

html_protocolo = get_html_content(URL_PROTOCOLO)

Requisição bem sucedida!


In [220]:
# Parse the HTML content
soup = BeautifulSoup(html_protocolo, 'html.parser')

In [222]:
# Recupera os dados das tabelas do site

table_dados = soup.find_all('table', class_='subListagem')
table_dados_interessado = table_dados[0]
table_dados_documentos = table_dados[1]
table_dados_movimentacoes = table_dados[2]

In [223]:
# Recupera dados do interessado do processo
for row in table_dados_interessado.find_all('tr'):
    data_cells = row.find_all('td', recursive=False)
    for cell in data_cells:
        print(cell.text.strip())

Servidor
1886865
ARMANDO SOARES SOUSA


In [224]:
# Recupera os dados basicos do processo
# Find the outer table
table_dados_basicos = soup.find('table', class_='formulario', style="width: 100%;")

# Extract data from each row
for i, row in enumerate(table_dados_basicos.find_all('tr')):
    data_cells = row.find_all('td', recursive=False)
    if i < 10:
      for cell in data_cells:
          print(cell.text.strip())

23111.052861/2021-71
Interno
22/12/2021 11:24
LUIS GUSTAVO DOS SANTOS VIANA
SOLICITACAO
RENOVAÇÃO DE AFASTAMENTO PARA QUALIFICAÇÃO (DOUTORADO)
OSTENSIVO
DIVISAO DE PROTOCOLO E DOCUMENTACAO/PRAD (11.00.15.08.03)
ARQUIVADO
22/12/2021


In [225]:
# Recupera os documentos do processo

lista_documentos = []
for row in table_dados_documentos.find_all('tr'):
    data_cells = row.find_all('td', recursive=False)
    my_link = ''
    lista_links = []
    for i, cell in enumerate(data_cells):
        if i == 0:
          tipo = cell.text.strip()
        elif i == 1:
          data = cell.text.strip()
        elif i == 2:
          origem = cell.text.strip()
        elif i == 3:
          natureza = cell.text.strip()
        if cell.find_all('a'):
          for link in cell.find_all('a'):
            lista_links.append(link['href'])
          my_link = lista_links[0]
        documento = DocumentoProcesso(tipo, data, origem, natureza, my_link)
    lista_documentos.append(documento)

In [204]:
for each in lista_documentos:
  print(each)

Tipo: DESPACHO, Data: 30/05/2022, Origem: CAP/SRH (11.00.15.05.01), Natureza: OSTENSIVO, Link: #
Tipo: REQUERIMENTO, Data: 22/12/2021, Origem: DIPROT/PRAD (11.00.15.08.03), Natureza: OSTENSIVO, Link: /public/downloadArquivo?idArquivo=4223938&key=1dc94c5c6e5a4558b5be94581c6e9a37
Tipo: DESPACHO, Data: 28/12/2021, Origem: DC/CCN (11.00.24.05), Natureza: OSTENSIVO, Link: #
Tipo: DESPACHO, Data: 28/12/2021, Origem: CCN (11.00.24), Natureza: OSTENSIVO, Link: #
Tipo: DESPACHO, Data: 04/01/2022, Origem: DC/CCN (11.00.24.05), Natureza: OSTENSIVO, Link: #
Tipo: PARECER DE RELATOR DE PROCESSO, Data: 12/01/2022, Origem: CCN (11.00.24), Natureza: OSTENSIVO, Link: /public/downloadArquivo?idArquivo=4237015&key=8b1c81e486c2e279d5e4a0222a83e57e
Tipo: DESPACHO, Data: 12/01/2022, Origem: CCN (11.00.24), Natureza: OSTENSIVO, Link: #
Tipo: DESPACHO, Data: 12/01/2022, Origem: GAB (11.00.01), Natureza: OSTENSIVO, Link: #
Tipo: DOCUMENTOS COMPROBATÓRIOS, Data: 05/01/2022, Origem: CCN (11.00.24), Natureza: OST

In [226]:
# Recupera as movimentacoes do processo

lista_movimentacoes = []
for row in table_dados_movimentacoes.find_all('tr'):
    data_cells = row.find_all('td', recursive=False)
    for i, cell in enumerate(data_cells):
      if i == 0:
        data_origem = cell.text.strip()
      elif i == 1:
        unidade_origem = cell.text.strip()
      elif i == 2:
        unidade_destino = cell.text.strip()
      elif i == 3:
        enviado_por = cell.text.strip()
      elif i == 4:
        recebido_em = cell.text.strip()
      elif i == 5:
        recebido_por = cell.text.strip()
      movimentacao = MovimentacaoProcesso(data_origem, unidade_origem, unidade_destino, enviado_por, recebido_em, recebido_por)
    lista_movimentacoes.append(movimentacao)

In [206]:
for movimentacao in lista_movimentacoes:
  print(movimentacao)
  print('--')

Data Origem: 30/05/2022 10:35, unidade origem: COORDENAÇÃO DE ADMINISTRACAO DE PESSOAL/SRH (11.00.15.05.01), unidade destino: DIVISÃO DE ARQUIVO E MICROFILMAGEM/PRAD (11.00.15.08.04), enviado por: hellyne, recebido em: 30/05/2022 10:50, recebido por: joaobliborio
--
Data Origem: 22/12/2021 11:23, unidade origem: DIVISAO DE PROTOCOLO E DOCUMENTACAO/PRAD (11.00.15.08.03), unidade destino: DEPARTAMENTO DE COMPUTAÇÃO/CCN (11.00.24.05), enviado por: luisgustavoviana, recebido em: 28/12/2021 12:35, recebido por: lucas.alexander
--
Data Origem: 28/12/2021 - 
												 Despacho Ordinatório  
												 
													- DEPARTAMENTO DE COMPUTAÇÃO/CCN (11.00.24.05), unidade origem: DIVISAO DE PROTOCOLO E DOCUMENTACAO/PRAD (11.00.15.08.03), unidade destino: DEPARTAMENTO DE COMPUTAÇÃO/CCN (11.00.24.05), enviado por: luisgustavoviana, recebido em: 28/12/2021 12:35, recebido por: lucas.alexander
--
Data Origem: , unidade origem: 28/12/2021 - 
												 Despacho Ordinatório  
												 
			

# Código Refatorado e Simplificado

In [233]:
def extract_data_from_table(table, headers):
    """Extrai dados de uma tabela HTML e retorna uma lista de dicionários."""
    data = []
    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if cells:  # Verifica se a linha contém dados
            row_data = dict(zip(headers, [cell.text.strip() for cell in cells]))
            data.append(row_data)
    return data

# Exemplo de uso:
table_dados = soup.find_all('table', class_='subListagem')
table_dados_interessado = table_dados[0]
table_dados_documentos = table_dados[1]
table_dados_movimentacoes = table_dados[2]

In [234]:
headers = ['Tipo', 'Identificador', 'Nome']
interessado_data = extract_data_from_table(table_dados_interessado, headers)

for pessoa in interessado_data:
    print(pessoa)

{'Tipo': 'Servidor', 'Identificador': '1886865', 'Nome': 'ARMANDO SOARES SOUSA'}


In [235]:
headers_documento = ['Tipo do Documento', 'Data do Documento', 'Origem', 'Natureza']
documentos = extract_data_from_table(table_dados_documentos, headers_documento)

for documento in documentos:
    print(documento)

{'Tipo do Documento': 'REQUERIMENTO', 'Data do Documento': '22/12/2021', 'Origem': 'DIPROT/PRAD (11.00.15.08.03)', 'Natureza': 'OSTENSIVO'}
{'Tipo do Documento': 'DESPACHO', 'Data do Documento': '28/12/2021', 'Origem': 'DC/CCN (11.00.24.05)', 'Natureza': 'OSTENSIVO'}
{'Tipo do Documento': 'DESPACHO', 'Data do Documento': '28/12/2021', 'Origem': 'CCN (11.00.24)', 'Natureza': 'OSTENSIVO'}
{'Tipo do Documento': 'DESPACHO', 'Data do Documento': '04/01/2022', 'Origem': 'DC/CCN (11.00.24.05)', 'Natureza': 'OSTENSIVO'}
{'Tipo do Documento': 'PARECER DE RELATOR DE PROCESSO', 'Data do Documento': '12/01/2022', 'Origem': 'CCN (11.00.24)', 'Natureza': 'OSTENSIVO'}
{'Tipo do Documento': 'DESPACHO', 'Data do Documento': '12/01/2022', 'Origem': 'CCN (11.00.24)', 'Natureza': 'OSTENSIVO'}
{'Tipo do Documento': 'DESPACHO', 'Data do Documento': '12/01/2022', 'Origem': 'GAB (11.00.01)', 'Natureza': 'OSTENSIVO'}
{'Tipo do Documento': 'DOCUMENTOS COMPROBATÓRIOS', 'Data do Documento': '05/01/2022', 'Origem'

In [236]:
headers_movimentacao =  ['Data Origem', 'Unidade Origem', 'Unidade Destino',	'Enviado Por',	'Recebido Em',	'Recebido Por']
movimentacoes = extract_data_from_table(table_dados_movimentacoes, headers_movimentacao)

for movimentacao in movimentacoes:
    print(movimentacao)

{'Data Origem': '22/12/2021 11:23', 'Unidade Origem': 'DIVISAO DE PROTOCOLO E DOCUMENTACAO/PRAD (11.00.15.08.03)', 'Unidade Destino': 'DEPARTAMENTO DE COMPUTAÇÃO/CCN (11.00.24.05)', 'Enviado Por': 'luisgustavoviana', 'Recebido Em': '28/12/2021 12:35', 'Recebido Por': 'lucas.alexander'}
{'Data Origem': '28/12/2021 - \n\t\t\t\t\t\t\t\t\t\t\t\t Despacho Ordinatório  \n\t\t\t\t\t\t\t\t\t\t\t\t \n\t\t\t\t\t\t\t\t\t\t\t\t\t- DEPARTAMENTO DE COMPUTAÇÃO/CCN (11.00.24.05)', 'Unidade Origem': '', 'Unidade Destino': '28/12/2021 - \n\t\t\t\t\t\t\t\t\t\t\t\t Despacho Ordinatório  \n\t\t\t\t\t\t\t\t\t\t\t\t \n\t\t\t\t\t\t\t\t\t\t\t\t\t- DEPARTAMENTO DE COMPUTAÇÃO/CCN (11.00.24.05)', 'Enviado Por': ''}
{'Data Origem': '', 'Unidade Origem': '28/12/2021 - \n\t\t\t\t\t\t\t\t\t\t\t\t Despacho Ordinatório  \n\t\t\t\t\t\t\t\t\t\t\t\t \n\t\t\t\t\t\t\t\t\t\t\t\t\t- DEPARTAMENTO DE COMPUTAÇÃO/CCN (11.00.24.05)', 'Unidade Destino': ''}
{'Data Origem': '28/12/2021 15:25', 'Unidade Origem': 'DEPARTAMENTO DE COMP